In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import datetime
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split, KFold
from wordcloud import WordCloud
from collections import Counter
import os
import xgboost as xgb
import lightgbm as lgb
from functools import reduce
import warnings
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
from lightgbm import plot_tree
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import STATUS_OK
from hyperopt.pyll.stochastic import sample
import gc
import math
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')


In [2]:
os.listdir('dataset-0510/')

['fields.pdf',
 'Result.xlsx',
 'submit_test.csv',
 'test.csv',
 'train.csv',
 '~$Result.xlsx',
 '下載 (1).png',
 '下載 (2).png',
 '下載 (3).png',
 '下載 (4).png',
 '下載.png']

In [3]:
train = pd.read_csv('dataset-0510/train.csv')
test  = pd.read_csv('dataset-0510/test.csv')
data = pd.concat([train, test], axis=0)
gc.collect()

16

In [4]:
def feature_process(df):
    
    orgin_columns = list(df.columns)
    
    #floor
    df.loc[df['txn_floor'].notna(), 'department'] = 1 
    df.loc[df['txn_floor'].isna(), 'department'] = 0
    df['avg_height_floor'] = df['txn_floor'] / df['total_floor'] 
    df['avg_height_floor'].fillna(1, inplace = True)
       
    #location
    df['location_2'] = df.apply(lambda x : int(str(x['city']) + str(x['town'])), axis=1)
    df['city'] = df['city'].astype('category')
    df['location_2'] = df['location_2'].astype('category')
    
    #date
    df['diff_between_txn_complete'] = df['txn_dt'] - df['building_complete_dt']
    df['rate_between_txn_complete'] = df['txn_dt'] / df['building_complete_dt']
    df['sum_between_txn_complete'] = df['txn_dt'] - df['building_complete_dt']
    df['year_between_txn_complete'] = round(df['diff_between_txn_complete'] / 365)
        
    #parking
    df['price_area'] = 1
    df.loc[df['parking_price'].notna(), 'price_area'] = 2
    df.loc[(df['parking_area'].notna()), 'price_area'] = 3
    
    
    df.loc[df['parking_way'] == 2, 'parking_area'] = 0
    df.loc[df['parking_way'] == 2, 'parking_price'] = 0
    
    #area
    df['sum_building_land'] = df['land_area'] + df['building_area']
    df['rate_building_land'] = df['land_area'] / df['building_area']
    
    df.loc[df['department'] == 1, 'building_area_every_floor'] = df.loc[df['department'] == 1, 'building_area']
    df.loc[df['department'] == 1, 'land_area_every_floor'] = df.loc[df['department'] == 1, 'land_area']
    
    df.loc[df['department'] == 0, 'building_area_every_floor'] = df.loc[df['department'] == 0, 'building_area'] / df.loc[df['department'] == 0, 'total_floor']
    df.loc[df['department'] == 0, 'land_area_every_floor'] = df.loc[df['department'] == 0, 'land_area'] / df.loc[df['department'] == 0, 'total_floor']
       
    
    #soical rate
    df['natural_diff'] = df['born_rate'] - df['death_rate']
    df['natural_rate'] = df['born_rate'] / df['death_rate']
    
    df['marry_diff'] = df['marriage_rate'] - df['divorce_rate']
    df['marry_rate'] = df['marriage_rate'] / df['divorce_rate']
    
    df['total_diff_sum'] = df['natural_rate'] + df['marry_diff']
    df['total_diff_diff'] = df['natural_rate'] - df['marry_diff']
    df['total_rate_diff'] = df['born_rate'] + df['marry_rate']
    
    df['positive_grow_rate'] = df['born_rate'] + df['marriage_rate']
    df['negative_grow_rate'] = df['death_rate'] + df['divorce_rate']
    df['tatal_rate_sum'] = df['born_rate'] + df['death_rate'] + df['born_rate'] + df['death_rate']
      
    #degree rate
    
    df['diff_doc_master'] = df['doc_rate'] - df['master_rate']
    df['diff_master_bachelor'] = df['master_rate'] - df['bachelor_rate']
    df['diff_bachelor_highsch'] = df['bachelor_rate'] - df['highschool_rate']
    df['diff_highsch_jobschool'] = df['highschool_rate'] - df['jobschool_rate']
    df['diff_jobschool_elesch'] = df['jobschool_rate'] - df['elementary_rate']
    
    df['all_degree'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate'] + df['highschool_rate'] + df['jobschool_rate'] + df['junior_rate'] + df['elementary_rate']
    df['junior_above_rate'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate'] + df['highschool_rate'] + df['jobschool_rate'] + df['junior_rate'] 
    df['jobschool_above_rate'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate'] + df['highschool_rate'] +  df['jobschool_rate']
    df['highschool_above_rate'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate'] + df['highschool_rate']
    df['bachelor_above_rate'] = df['doc_rate'] + df['master_rate'] + df['bachelor_rate']
    df['master_above_rate'] = df['doc_rate'] + df['master_rate'] 
    
    #MIN
    MIN_cols = [i for i in orgin_columns if '_MIN' in i]
    df['MIN_sum']   = df[MIN_cols].sum(axis=1)
    df['MIN_mean']   = df[MIN_cols].mean(axis=1)
    df['MIN_median'] = df[MIN_cols].median(axis=1)
    df['MIN_std']    = df[MIN_cols].std(axis=1)
    #df['MIN_skew']   = df[MIN_cols].skew(axis=1)
    df['MIN_max']    = df[MIN_cols].max(axis=1)
    df['MIN_min']    = df[MIN_cols].min(axis=1)
    df['MIN_diff'] = df['MIN_max'] - df['MIN_min']
    
    rank_df = df[MIN_cols].rank(axis =1)
    rank_df = rank_df.add_prefix('Rank_')
    df = pd.concat([df, rank_df], axis=1)
    
    for stat in ['_sum', '_median']:
        compute_cols = [i for i in df.columns if stat in i and 'MIN' not in i and 'All' in i]
        for i in range(len(compute_cols)-1):
            former = compute_cols[i]
            latter = compute_cols[i+1]
            df[latter + '_' + former + '_diff']       = df[latter] - df[former]
            df[latter + '_' + former + '_dense_diff'] = df[latter] / int(latter.split('_')[1]) - df[former] / int(former.split('_')[1])
            df[latter + '_' + former + '_diff_dense'] = (df[latter] - df[former]) / (int(latter.split('_')[1]) - int(former.split('_')[1]))    
    
    #Build_case
    cols = ['city', 'town', 'village', 'building_type', 'building_use', 'total_floor', 'XIV_MIN', 'building_complete_dt']
    df.loc[df[cols].duplicated(keep = False), 'Build_case'] = 0
    df.loc[~df[cols].duplicated(keep = False), 'Build_case'] = 1
    df['Build_case_count'] = df.groupby(cols)['total_price'].transform('count')
    
    #10 50 100 250 500 1000 5000 10000
    
    df['diff_500_50'] = df['N_500'] - df['N_50']
    df['diff_1000_500'] = df['N_1000'] - df['N_500']
    df['diff_5000_1000'] = df['N_5000'] - df['N_1000']
    df['diff_10000_5000'] = df['N_10000'] - df['N_5000']
    df['rate_500_50'] = df['N_500'] / df['N_50']
    df['rate_1000_500'] = df['N_1000'] / df['N_500']
    df['rate_5000_1000'] = df['N_5000'] / df['N_1000']
    df['rate_10000_5000'] = df['N_10000'] / df['N_5000']
    
    All_10    = [i for i in orgin_columns if i.endswith('_10') and 'index' not in i and 'N' not in i ]
    All_50    = [i for i in orgin_columns if i.endswith('_50') and 'index' not in i and 'N' not in i]
    All_100   = [i for i in orgin_columns if i.endswith('_100') and 'index' not in i and 'N' not in i ]
    All_250   = [i for i in orgin_columns if i.endswith('_250') and 'index' not in i and 'N' not in i ]
    All_500   = [i for i in orgin_columns if i.endswith('_500') and 'index' not in i and 'N' not in i ]
    All_1000  = [i for i in orgin_columns if i.endswith('_1000') and 'index' not in i and 'N' not in i ]
    All_5000  = [i for i in orgin_columns if i.endswith('_5000') and 'index' not in i and 'N' not in i ]
    All_10000 = [i for i in orgin_columns if i.endswith('_10000') and 'index' not in i and 'N' not in i ]
    All_dict = {'All_10' : All_10, 'All_50' : All_50, 'All_100' : All_100, 'All_250' : All_250, 'All_1000' : All_1000, 'All_5000' : All_5000, 'All_10000' : All_10000}

    
    for name, col in All_dict.items():
        df[name + '_sum']     = df[col].sum(axis=1)
        df[name + '_mean']    = df[col].mean(axis=1)
        df[name + '_median']  = df[col].median(axis=1)
        df[name + '_std']     = df[col].std(axis=1)
        #df[name + '_skew']    = df[col].skew(axis=1)
        #df[name + '_max']     = df[col].max(axis=1)
        #df[name + '_min']     = df[col].min(axis=1)
       
    All = [All_10, All_50, All_100, All_250, All_1000, All_5000, All_10000]
    for i in range(len(All) - 1):
        former = All[i]
        latter = All[i+1]
        for j in range(len(All[0])):
            df[latter[j] + '_' + former[j] + '_diff'] = df[latter[j]] - df[former[j]]
            df[latter[j] + '_' + former[j] + '_dense_diff'] = df[latter[j]] / int(latter[j].split('_')[1]) - df[former[j]] / int(former[j].split('_')[1])
            df[latter[j] + '_' + former[j] + '_diff_dense'] = (df[latter[j]] - df[former[j]]) / (int(latter[j].split('_')[1]) - int(former[j].split('_')[1]))
    
    #interection

    inter_cols = ['building_type', 'building_use', 'building_material', 'parking_way']
    for i in range(4):
        for j in range(4):
            if j > i:
                df['inter_btw_' + inter_cols[i] +'_' + inter_cols[j]] = df.apply(lambda x : str(x[inter_cols[i]]) + str(x[inter_cols[j]]), axis=1)
                df['inter_btw_' + inter_cols[i] +'_' + inter_cols[j]] = df['inter_btw_' + inter_cols[i] +'_' + inter_cols[j]].astype('category')
     
    #frequency encoding
    
    df['city_size'] = df.groupby('city')['building_id'].transform('size')
    df['city_building_size'] = df.groupby(['city', 'building_type'])['building_id'].transform('size')
    df['city_town_size'] = df.groupby(['city', 'town'])['building_id'].transform('size')
    df['city_building_type_frequency'] = df['city_building_size'] / df['city_size']
    df['city_town_frequency'] = df['city_town_size'] / df['city_size']
    df.drop(['city_size', 'city_building_size', 'city_town_size'], axis = 1)

    df['location_2_size'] = df.groupby('location_2')['building_id'].transform('size')
    df['location_2_building_size'] = df.groupby(['location_2', 'building_type'])['building_id'].transform('size')
    df['location_2_building_type_frequency'] = df['location_2_building_size'] / df['location_2_size']
    df.drop(['city_size', 'location_2_building_size'], axis = 1)
    
    df['building_type_size'] = df.groupby('building_type')['building_id'].transform('size')
    df['building_type_building_use_size'] = df.groupby(['building_type', 'building_use'])['building_id'].transform('size')
    df['building_type_building_use_frequency'] = df['building_type_building_use_size'] / df['building_type_size']
    df.drop(['building_type_size', 'building_type_building_use_size'], axis = 1)
    

    
    #one-hot
    
    df = pd.concat([df, pd.get_dummies(df['parking_way'], prefix = 'parking_way_')], axis=1)
    df = pd.concat([df, pd.get_dummies(df['building_type'], prefix = 'building_type_')], axis=1)
    df = pd.concat([df, pd.get_dummies(df['building_material'], prefix = 'building_material_')], axis=1)
    df = pd.concat([df, pd.get_dummies(df['building_use'], prefix = 'building_use_')], axis=1)
    
    #groupby encoding
    
    category_cols  = ['building_type', 'building_use', 'city']
    
    numerical_cols = ['building_area', 'land_area', 'building_complete_dt'] + MIN_cols
    statistics = ['mean', 'median', 'max']
    for category in category_cols:
        for numerical in numerical_cols:
            for stat in statistics:
                df[numerical + '_' + stat + '_gb_' + category] = df.groupby([category])[numerical].transform(stat)
                df[numerical + '_diff_' + stat + '_gb_' + category] = df[numerical] - df[numerical + '_' + stat + '_gb_' + category]
                #df.drop([numerical + '_' + stat + '_gb_' + category], inplace = True, axis = 1) 
      
    for category in ['building_type', 'building_use']:
        for numerical in MIN_cols:
            for stat in ['mean', 'median', 'max', 'min']:
                df[numerical + '_' + stat + '_gb_' + 'city' + '_' + category] = df.groupby(['city'] + [category])[numerical].transform(stat)
                df[numerical + '_diff_' + stat + '_gb_' + 'city' + '_' + category] = df[numerical] - df[numerical + '_' + stat + '_gb_' + 'city' + '_' + category]
     
    
    # Catch min_dsi category
    temp = df[MIN_cols]
    result = [[],[],[],[],[]]
    n = len(cols)
    
    for idx, row in tqdm(temp.iterrows()):
        dic = {}
        for i in range(n):
            dic[row.index[i]] = row[i]
        sorted_d = sorted(dic.items(), key=lambda kv: kv[1])
        for i in range(5):
            result[i].append(sorted_d[i][0])
            
    for i in range(5):
        col_name = 'MIN_category_' + str(i+1)
        df[col_name] = result[i]
    
    le = preprocessing.LabelEncoder() 
    df['MIN_category_1'] = le.fit_transform(df['MIN_category_1'])
    df['MIN_category_2'] = le.fit_transform(df['MIN_category_2'])
    df['MIN_category_3'] = le.fit_transform(df['MIN_category_3'])
    df['MIN_category_4'] = le.fit_transform(df['MIN_category_4'])
    df['MIN_category_5'] = le.fit_transform(df['MIN_category_5'])
    
    
    #Impute missing value
    df['village_income_median'] = df.groupby(['city', 'town'])['village_income_median'].transform(lambda x : x .fillna(x.median()))
    df['parking_price'] = df.groupby(['building_type', 'parking_way'])['parking_price'].transform(lambda x : x.median())
    
    #useless cols
    drop_cols = [i for i in orgin_columns if i.endswith('_rate')]
    df.drop(drop_cols, axis = 1, inplace = True)
    df.drop([i for i in orgin_columns if np.sum(train[i]) == 60000 and 'index' in i], axis = 1, inplace = True)
    df.drop(['building_use', 'building_material'], axis = 1, inplace = True)
    return df

In [5]:
FE_data = feature_process(data)
FE_train = FE_data[:60000]
FE_test = FE_data[60000:]
FE_train.to_csv('FE_train.csv', index = False)
FE_test.to_csv('FE_test.csv', index = False)
len(FE_train.columns), len(FE_test.columns)

70000it [00:06, 11042.75it/s]


(1123, 1123)

In [6]:
FE_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Columns: 1123 entries, III_10 to MIN_category_5
dtypes: category(8), float64(794), int32(5), int64(288), object(1), uint8(27)
memory usage: 499.4+ MB
